In [1]:
from utils import db
from utils import s3

In [4]:
# 1. Create a a file:
# col:
# product_name varchar(20),
# manufacturingcost decimal(5,2)
# For each product build record using python script.
# 'iphon11','iphone12','iphone13','iphoneSE','IpadMax','IpadMini','laptop256','Macbook512'],
# 'Samsung': ['galaxy10','galaxy11','galaxy12','galaxy13','watch320','watch340'

import random

product_costs = {}

product_names = ['iphon11','iphone12','iphone13','iphoneSE','IpadMax','IpadMini','laptop256','Macbook512', 
    'galaxy10','galaxy11','galaxy12','galaxy13','watch320','watch340', 'Nk320','Nk400','Nk500']

f = open('data/cards_ingest/product_cost.csv', 'w')
f.write('product_name' + ',' + 'manufacturing_cost\n')
for p in product_names:
    f.write(p + ',' + str(round(random.uniform(100.00, 500.00),2)) + '\n')
f.close()

In [1]:
# 2. Load data into s3 in separate folder (order data must be created earlier in class project 3).
s3.upload_file('data/cards_ingest/product_cost.csv', 'quintrix-spearscjs', 'cards_ingest/product_cost.csv')
s3.upload_file('data/cards_ingest/order_data_20230401.csv', 'quintrix-spearscjs', 'cards_ingest/order_data/order_data_20230401.csv')

In [5]:
# 3. Use copy command  to load data into redshift by.
db.do_query_file('sql/create/ct_order.sql', [])
db.do_query_file('sql/create/ct_product_cost.sql', [])

args = ['vw.order_product_cost', 's3://views-quintrix-spearscjs/order/vw_order_product_cost.csv',
    db.redshift_iam, ',']
res = db.do_query_file('sql/query_templates/copy.sql', args)


sql/create/ct_order.sql done.
sql/create/ct_product_cost.sql done.


In [13]:
# 4  create a view by joining order and product_cost.
query = """ 
    CREATE SCHEMA IF NOT EXISTS vw; 
"""
db.do_query(query, [])

db.do_query_file('sql/create/view/ct_order_product_cost.sql', [])

sql/create/view/ct_order_product_cost.sql done.


In [7]:
# 5. Use boto3 to create bucket then list the buckets
s3.create_bucket('views-quintrix-spearscjs')

print(s3.client.list_buckets(), '\n')

if(s3.bucket_exists('views-quintrix-spearscjs')):
    print("SUCCESS")
else:
    print("FAILURE")

{'ResponseMetadata': {'RequestId': 'YX0EG4BD5EBWF10R', 'HostId': '4aDgYVa779SkIcwOgiEE3DUy47LlhCM3ntA2W7RWv6PPQeseLsWVfY7io/jjR4UL9+lwxBGf2sg=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '4aDgYVa779SkIcwOgiEE3DUy47LlhCM3ntA2W7RWv6PPQeseLsWVfY7io/jjR4UL9+lwxBGf2sg=', 'x-amz-request-id': 'YX0EG4BD5EBWF10R', 'date': 'Tue, 17 Jan 2023 18:45:29 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'quintrix-spearscjs', 'CreationDate': datetime.datetime(2023, 1, 9, 19, 37, 25, tzinfo=tzutc())}, {'Name': 'test-spearscjs', 'CreationDate': datetime.datetime(2023, 1, 12, 21, 39, 7, tzinfo=tzutc())}, {'Name': 'views-quintrix-spearscjs', 'CreationDate': datetime.datetime(2023, 1, 12, 19, 14, 27, tzinfo=tzutc())}, {'Name': 'views-quintrix-test1', 'CreationDate': datetime.datetime(2023, 1, 17, 18, 8, 9, tzinfo=tzutc())}], 'Owner': {'DisplayName': 'spearscjs', 'ID': '1b7f5da4c80f4cc0884c213769d24009e307959b

In [6]:
# 6. Unload data from view into s3 bucket (this has to be separate bucket then created earlier) (not doing yet)

# csv, header, allow overwrite, optinal? (null as)

s3.empty_bucket('views-quintrix-spearscjs')
query = 'select * from vw.order_product_cost'
args = [query, 's3://views-quintrix-spearscjs/order/vw_order_product_cost.csv',
    db.redshift_iam, ',']

res = db.do_query_file('sql/query_templates/unload.sql', args)

sql/query_templates/unload.sql done.


In [1]:
# 7. Create a new view by taking all the records where you have profit. Profit is sales amount -manufacturingcost. So add all the fields from order table
# and cost from product_cost.
db.do_query_file('sql/create/view/ct_order_profit.sql', [])

sql/create/view/ct_order_profit.sql done.


In [4]:
# 8. Create col as profit_group the values are [ if profit percenatge is more 10% the "Bumper profit", it is 0 to 10% then "Marginal profit"
# if it is -5% to 0% then "Loss making" else "Bumper Loss"

query = """ 
DROP VIEW IF EXISTS vw.order_profit;
"""
db.do_query(query, [])

db.do_query_file('sql/create/view/ct_order_profit.sql', [])

sql/create/view/ct_order_profit.sql done.


In [5]:
# 9. Unload the view into a s3 file.
query = 'select * from vw.order_profit'
args = [query, 's3://views-quintrix-spearscjs/view/vw_order_profit.csv',
    db.redshift_iam, ',']

db.do_query_file('sql/query_templates/unload.sql', args)

sql/query_templates/unload.sql done.


In [7]:
# 10. Create another view to show total profit by each brand_name,year-mth cd (ex:202201,202203). Do a rank by the total profit (desc) and
# amount of product sold (Asc)
query = """ 
    DROP VIEW IF EXISTS vw.order_total_profit;
"""
db.do_query(query, [])
db.do_query_file('sql/create/view/ct_profit_rank.sql', [])

sql/create/view/ct_profit_rank.sql done.


In [15]:
# 11. unload data to a CSV file wtih header and pipe delimited
query = 'select * from vw.profit_rank'
args = [query, 's3://views-quintrix-spearscjs/order/vw_profit_rank.csv',
    db.redshift_iam, '|']

db.do_query_file('sql/query_templates/unload.sql', args)

,brand_name,year_month,total_profit,num_sold,profit_rank,num_sold_rank
0,Samsung,2018-5,189063.14,721,1,3
1,Apple,2018-5,188758.19,743,2,2
2,Nokia,2018-5,134938.57,734,3,1
3,Apple,2018-6,193124.29,670,1,3
4,Samsung,2018-6,169570.16,662,2,2
...,...,...,...,...,...,...
67,Apple,2020-3,178996.55,682,2,2
68,Nokia,2020-3,136177.53,706,3,1
69,Samsung,2020-4,191462.48,704,1,3
70,Apple,2020-4,179230.93,673,2,2
